In [ ]:
using DynamicalSystems
using ChaosTools
using LinearAlgebra
using Random

In [ ]:
OPS = load("data/kse_operators_L22.jld2");

In [ ]:
@inline @inbounds function kse(x, p, t)
    A = p[1]
    H = p[2]
    N = size(A,1)
    xdot = A*x + H * kron(x,x)
    return SVector{N}(xdot...)
end

@inline @inbounds function kse_jacobian(x, p, t)
    A = p[1]
    H = p[2]
    N = size(A,1)
    J = @SMatrix A + H * kron(1.0I(N), x) + H * kron(x, 1.0I(N))
    return J
end